<a href="https://colab.research.google.com/github/DancheG14/Hack2023_CheckOut/blob/Andrey/main_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from keras.utils import to_categorical
import numpy as np
import pandas as pd
import docx
from docx import Document

In [31]:
#!wget "https://www.dropbox.com/s/2osnp0i9k8i4gbs/docx.zip"
#!unzip -q docx.zip
"""
dir = '/content/docx/'
fileText = []

for fileIndex in range(9999):
     name = str(fileIndex) + '.docx' 
     doctext = []
     doc = docx.Document(dir + name)
     for paragraph in doc.paragraphs:
         if paragraph.text != "":
             doctext.append([paragraph.text.replace("\n","")])  
     fileText.append(doctext)
"""

In [3]:
df = pd.read_csv('df_all.csv')
df2 = df.copy()
df2 = df.drop('id', axis=1)

In [4]:
df2.head(1)

,Task1.task_text,Task1.task_responsibles_people,Task1.task_responsibles_groups,Task1.task_deadline,Global_supervisor,Global_deadline,Task2.task_text,Task2.task_responsibles_people,Task2.task_responsibles_groups,Task2.task_deadline,...,Task7.task_responsibles_groups,Task7.task_deadline,Task8.task_text,Task8.task_responsibles_people,Task8.task_responsibles_groups,Task8.task_deadline,Task9.task_text,Task9.task_responsibles_people,Task9.task_responsibles_groups,Task9.task_deadline
0,Закупить книги. Срок завершения: 19 апр 15.,Ответственность за исполнение распоряжения воз...,8,"['19 апр 15', 1429383600.0]",['Ответственность за исполнение распоряжения в...,"['11 апреля 2013 года', 1365616800.0]",0,0,0.0,0,...,0.0,0,0,0,0.0,0,0,0,0.0,0


In [5]:
y = df2['Task1.task_responsibles_groups']
df2 = df2.drop(['Task1.task_responsibles_groups','Task2.task_responsibles_groups','Task3.task_responsibles_groups','Task4.task_responsibles_groups',
            'Task5.task_responsibles_groups','Task6.task_responsibles_groups','Task7.task_responsibles_groups','Task8.task_responsibles_groups', 'Task9.task_responsibles_groups'], axis = 1)

In [6]:
df2.shape, y.shape

((9999, 29), (9999,))

In [26]:
num_words = 100000
max_review_len = 66
tokenizer = Tokenizer(num_words=num_words)

In [35]:
#tokenizer.fit_on_texts(str(fileText))

In [27]:
for i in range(29):
  tokenizer.fit_on_texts(df2.iloc[:,i])

In [28]:
df3 = df2.copy()

In [29]:
for i in range(29):
  df3.iloc[:,i] = tokenizer.texts_to_sequences(df3.iloc[:,i])


In [30]:
df3

,Task1.task_text,Task1.task_responsibles_people,Task1.task_deadline,Global_supervisor,Global_deadline,Task2.task_text,Task2.task_responsibles_people,Task2.task_deadline,Task3.task_text,Task3.task_responsibles_people,...,Task6.task_deadline,Task7.task_text,Task7.task_responsibles_people,Task7.task_deadline,Task8.task_text,Task8.task_responsibles_people,Task8.task_deadline,Task9.task_text,Task9.task_responsibles_people,Task9.task_deadline
0,"[188, 557, 27, 102, 145, 161, 138]","[78, 2, 21, 8, 9, 6, 5, 22, 4, 30, 31, 7, 118,...","[206, 161, 180, 4049, 1]","[79, 2, 21, 8, 9, 6, 5, 22, 4, 30, 31, 7, 118,...","[219, 164, 123, 46, 1573, 1]",[1],[1],[1],[1],[1],...,[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]
1,"[40, 218, 599, 37, 419, 283, 229, 261, 181, 27...","[78, 2, 21, 8, 9, 6, 5, 54, 4, 55, 59, 239, 24...","[202, 156, 177, 2382, 1]","[79, 2, 21, 8, 9, 6, 5, 54, 4, 55, 59, 239, 24...","[214, 71, 123, 46, 2755, 1]","[53, 184, 653, 162, 427, 75, 654, 655, 656, 81...","[10, 3, 15, 16, 9, 6, 150, 57, 241, 341]","[211, 160, 179, 4008, 1]","[504, 20, 281, 586, 587, 23, 289, 12, 290, 349...","[10, 3, 14, 8, 42, 2, 67, 54, 4, 55, 59, 776, ...",...,[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]
2,"[40, 505, 111, 113, 37, 305, 288, 27, 25, 49, ...","[10, 3, 14, 8, 42, 2, 823, 243, 251, 824, 969]","[202, 62, 176, 1244, 1]","[24, 3, 14, 8, 42, 2, 823, 243, 251, 824, 971,...","[197, 166, 131, 29, 13, 3054, 1]","[40, 505, 111, 113, 37, 305, 288, 66, 27, 25, ...","[18, 2, 21, 17, 16, 11, 262, 243, 251, 263, 364]","[208, 165, 141, 46, 1293, 1]","[40, 218, 461, 149, 462, 20, 41, 463, 464, 351...","[2, 10, 3, 14, 8, 11, 18, 271, 272, 373]",...,"[195, 105, 175, 2027, 1]",[1],[1],[1],[1],[1],[1],[1],[1],[1]
3,"[389, 578, 579, 41, 27, 100, 52, 101, 26, 245,...","[10, 3, 15, 17, 8, 48, 2, 82]","[205, 68, 115, 29, 13, 3755, 1]","[24, 3, 15, 17, 8, 48, 2, 83, 80, 25, 9, 6, 5,...","[209, 62, 174, 1272, 1]","[40, 393, 615, 75, 23, 616, 456, 27, 102, 127,...","[65, 25, 9, 6, 5, 108, 4, 36, 72, 19, 12]","[207, 99, 179, 1128, 1]","[425, 426, 256, 6, 487, 181, 85, 32, 86, 49, 2...","[10, 3, 15, 17, 8, 48, 2, 82]",...,[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]
4,"[284, 558, 6, 34, 70, 345, 323, 92, 93, 51, 23...","[10, 3, 15, 17, 8, 48, 2, 82]","[211, 153, 115, 29, 13, 4050, 1]","[24, 3, 15, 17, 8, 48, 2, 83, 47, 2, 21, 17, 1...","[206, 110, 340, 1055, 1]","[381, 256, 6, 367, 546, 66, 27, 25, 52, 26, 24...","[18, 2, 21, 17, 16, 11, 5, 28, 4, 36, 140, 64,...","[205, 99, 340, 1741, 1]",[1],[1],...,[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,"[53, 184, 653, 162, 427, 75, 654, 655, 656, 92...","[18, 2, 21, 17, 16, 11, 5, 28, 4, 55, 59, 231,...","[198, 71, 176, 986, 1]","[47, 2, 21, 17, 16, 11, 5, 28, 4, 55, 59, 231,...","[219, 165, 130, 46, 1694, 1]","[53, 643, 405, 181, 41, 27, 100, 52, 101, 26, ...","[2, 10, 3, 14, 8, 11, 18, 246, 121, 227, 191, ...","[200, 105, 330, 2936, 1]",[1],[1],...,[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]
9995,"[98, 681, 37, 432, 55, 85, 32, 86, 49, 26, 139...","[18, 2, 21, 17, 16, 11, 124, 125, 126, 60, 12]","[194, 173, 129, 29, 13, 3728, 1]","[47, 2, 21, 17, 16, 11, 124, 125, 126, 60, 12,...","[220, 171, 146, 46, 2171, 1]","[425, 426, 256, 37, 471, 711, 12, 151, 712, 71...","[2, 10, 3, 14, 8, 11, 18, 5, 108, 4, 36, 72, 1...","[200, 68, 123, 29, 13, 1470, 1]",[1],[1],...,[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]
9996,"[188, 415, 416, 435, 388, 111, 113, 90, 50, 70...","[10, 3, 14, 8, 42, 2, 67, 54, 4, 55, 59, 776, ...","[214, 156, 116, 46, 1552, 1]","[24, 3, 14, 8, 42, 2, 67, 54, 4, 55, 59, 776, ...","[205, 68, 177, 1979, 1]","[188, 415, 416, 686, 20, 687, 688, 37, 112, 27...","[18, 2, 43, 3, 14, 44, 38, 11, 150, 57, 241, 341]","[193, 171, 117, 46, 1607, 1]","[53, 184, 653, 162, 427, 75, 654, 655, 656, 81...","[10, 3, 15, 17, 16, 9, 6, 5, 22, 4, 55, 59, 72...",...,"[198, 163, 186, 4437, 1]","[66, 612, 375, 65, 92, 93, 51, 23, 21, 32, 91,...","[65, 25, 9, 6, 246, 121,

In [31]:
#скрипт для получения максимальной длины вектора в датасете
max = 0

for i in range(29):
  p = len(np.max(df3.iloc[:,i]))
  if p > max:
    max = p
print(max)

66


In [32]:
test = pad_sequences(df3.iloc[:,i], maxlen = max)
test[3]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int32)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(test,
                                                    y,
                                                    
                                                    test_size=0.1,
                                                    random_state=50)
X_train.shape, y_train.shape

nb_classes = 8
y_train = utils.to_categorical(y_train - 1, nb_classes)
y_test = utils.to_categorical(y_test - 1, nb_classes)

In [34]:
model_lstm = Sequential()
model_lstm.add(Embedding(num_words, 32, input_length=max))
model_lstm.add(LSTM(16))
model_lstm.add(Dense(8, activation='softmax'))

model_lstm.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

history_lstm = model_lstm.fit(X_train, 
                              y_train, 
                              epochs=10,
                              batch_size=128,
                              validation_split=0.1,
                              )

Epoch 1/10
64/64 [==============================] - 5s 46ms/step - loss: 1.9469 - accuracy: 0.2006 - val_loss: 1.8826 - val_accuracy: 0.1789
Epoch 2/10
64/64 [==============================] - 2s 35ms/step - loss: 1.8862 - accuracy: 0.2066 - val_loss: 1.8767 - val_accuracy: 0.1789
Epoch 3/10
64/64 [==============================] - 2s 34ms/step - loss: 1.8851 - accuracy: 0.2025 - val_loss: 1.8825 - val_accuracy: 0.1722
Epoch 4/10
64/64 [==============================] - 2s 35ms/step - loss: 1.8844 - accuracy: 0.2088 - val_loss: 1.8747 - val_accuracy: 0.1911
Epoch 5/10
64/64 [==============================] - 2s 34ms/step - loss: 1.8829 - accuracy: 0.2140 - val_loss: 1.8748 - val_accuracy: 0.1900
Epoch 6/10
41/64 [==================>...........] - ETA: 0s - loss: 1.8822 - accuracy: 0.2262

KeyboardInterrupt: ignored